<a href="https://colab.research.google.com/github/rahiakela/machine-learning-research-and-practice/blob/main/hands-on-machine-learning-with-scikit-learn-keras-and-tensorflow/12-custom-models-and-training-with-tensorflow/05_tensorflow_functions_and_graphs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## TensorFlow Functions and Graphs

In fact, 95% of the use cases you will encounter will not require anything other than `tf.keras` and `tf.data`.

But now it’s time to dive deeper into TensorFlow
and take a look at its lower-level Python API. This will be useful when you need extra
control to write custom loss functions, custom metrics, layers, models, initializers,
regularizers, weight constraints, and more. 

You may even need to fully control the
training loop itself, for example to apply special transformations or constraints to the
gradients (beyond just clipping them) or to use multiple optimizers for different parts
of the network.

TensorFlow’s API revolves around tensors, which flow from operation to operation—hence the name TensorFlow.

A tensor is very similar to a NumPy ndarray: it is usually
a multidimensional array, but it can also hold a scalar (a simple value, such as 42).
These tensors will be important when we create custom cost functions, custom metrics,
custom layers, and more, so let’s see how to create and manipulate them.



##Setup

In [1]:
import sys
import sklearn
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow import keras

from tqdm.notebook import trange
from collections import OrderedDict

import numpy as np
import os
import time

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

## Loading Dataset

Let's start by loading and preparing the California housing dataset. 

In [2]:
housing = fetch_california_housing()

x_train_full, x_test, y_train_full, y_test = train_test_split(housing.data, housing.target.reshape(-1, 1), random_state=42)
x_train, x_valid, y_train, y_valid = train_test_split(x_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_valid_scaled = scaler.transform(x_valid)
x_test_scaled = scaler.transform(x_test)

##TensorFlow Function

Let’s start with a trivial function that computes the cube of its input.

In [3]:
def cube(x):
  return x ** 3

In [4]:
cube(2)

8

In [5]:
cube(tf.constant(2.0))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

Now, let’s use `tf.function()` to convert this Python function to a TensorFlow
Function:

In [6]:
tf_cube = tf.function(cube)
tf_cube

In [7]:
tf_cube(2)

<tf.Tensor: shape=(), dtype=int32, numpy=8>

In [8]:
tf_cube(tf.constant(2.0))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

Under the hood, `tf.function()` analyzed the computations performed by the `cube()` function and generated an equivalent computation graph!

Alternatively, we could have used
`tf.function` as a decorator; this is actually more common.

In [9]:
@tf.function
def my_cube(x):
  return x ** 3

The original Python function is still available via the TF Function’s `python_function` attribute, in case you ever need it:

In [10]:
my_cube.python_function(2)

8

TensorFlow optimizes the computation graph, pruning unused nodes, simplifying
expressions (e.g., 1 + 2 would get replaced with 3), and more. Once the optimized
graph is ready, the TF Function efficiently executes the operations in the graph, in the
appropriate order (and in parallel when it can).

As a result, a TF Function will usually
run much faster than the original Python function, especially if it performs complex
computations.

**Most of the time you will not really need to know more than that:
when you want to boost a Python function, just transform it into a TF Function.**

##Concrete Functions

TF Functions are polymorphic, meaning they support inputs of different types.

Every time you call a TF Function with a new combination of input types or shapes, it
generates a new concrete function, with its own graph specialized for this particular
combination. Such a combination of argument types and shapes is called an input signature.

In [11]:
concrete_function = my_cube.get_concrete_function(tf.constant(2.0))
concrete_function.graph

In [12]:
concrete_function(tf.constant(2.0))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

In [13]:
concrete_function(tf.constant(5.0))

<tf.Tensor: shape=(), dtype=float32, numpy=125.0>

##Function Definitions and Graphs

You can access a concrete function’s computation graph using the graph attribute.

In [14]:
ops = concrete_function.graph.get_operations()
ops

[<tf.Operation 'x' type=Placeholder>,
 <tf.Operation 'pow/y' type=Const>,
 <tf.Operation 'pow' type=Pow>,
 <tf.Operation 'Identity' type=Identity>]

Let’s get the list of inputs and outputs of the power operation.

In [15]:
pow_ops = ops[2]
list(pow_ops.inputs)

[<tf.Tensor 'x:0' shape=() dtype=float32>,
 <tf.Tensor 'pow/y:0' shape=() dtype=float32>]

In [16]:
pow_ops.outputs

[<tf.Tensor 'pow:0' shape=() dtype=float32>]

Note that each operation has a name.

In [17]:
concrete_function.graph.get_operation_by_name("x")

<tf.Operation 'x' type=Placeholder>

In [18]:
concrete_function.graph.get_tensor_by_name("Identity:0")

<tf.Tensor 'Identity:0' shape=() dtype=float32>

The concrete function also contains the function definition which includes the function’s signature.

In [19]:
concrete_function.function_def.signature

name: "__inference_my_cube_25"
input_arg {
  name: "x"
  type: DT_FLOAT
}
output_arg {
  name: "identity"
  type: DT_FLOAT
}

##Function Tracing

In [20]:
@tf.function
def tf_cube(x):
  print("x: ", x)
  return x ** 3

In [21]:
result = tf_cube(tf.constant(2.0))

x:  Tensor("x:0", shape=(), dtype=float32)


In [22]:
result

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

It has a
shape and a data type, but no value. Plus it has a name ("x:0"). This is because the
`print()` function is not a TensorFlow operation, so it will only run when the Python
function is traced, which happens in graph mode, with arguments replaced with symbolic
tensors (same type and shape, but no value). 

Since the `print()` function was not
captured into the graph, the next times we call `tf_cube()` with float32 scalar tensors,
nothing is printed:

In [23]:
result = tf_cube(tf.constant(3.0))

In [24]:
result = tf_cube(tf.constant(5.0))

But if we call `tf_cube()` with a tensor of a different type or shape, or with a new
Python value, the function will be traced again.

In [25]:
result = tf_cube(2)  # new Python value: trace!

x:  2


In [26]:
result = tf_cube(4)   # new Python value: trace!

x:  4


In [27]:
result = tf_cube(tf.constant([[1.0, 2.0]]))   # New shape: trace!

x:  Tensor("x:0", shape=(1, 2), dtype=float32)


In [28]:
result = tf_cube(tf.constant([[3.0,4.0], [5.0, 6.0]]))   # New shape: trace!

x:  Tensor("x:0", shape=(2, 2), dtype=float32)


In [29]:
result = tf_cube(tf.constant([[7.0,8.0], [9.0, 10.0]]))   # Same shape: no trace!

In some cases, you may want to restrict a TF Function to a specific input signature.

For example, suppose you know that you will only ever call a TF Function with
batches of 28 × 28–pixel images, but the batches will have very different sizes. You
may not want TensorFlow to generate a different concrete function for each batch
size, or count on it to figure out on its own when to use None. 

In this case, you can
specify the input signature like this:

In [30]:
@tf.function(input_signature=[tf.TensorSpec([None, 28, 28], tf.float32)])
def shrink(images):
  return images[:, ::2, ::2]  # drop half the rows and columns

This TF Function will accept any float32 tensor of shape `[*, 28, 28]`, and it will reuse
the same concrete function every time:

In [31]:
img_batch_1 = tf.random.uniform(shape=[100, 28, 28])
img_batch_2 = tf.random.uniform(shape=[50, 28, 28])

preprocessed_images = shrink(img_batch_1)   # Works fine. Traces the function.
preprocessed_images = shrink(img_batch_2)   # Reuses the same concrete function

However, if you try to call this TF Function with a Python value, or a tensor of an
unexpected data type or shape, you will get an exception:

In [32]:
img_batch_3 = tf.random.uniform(shape=[2, 2, 2])
try:
  preprocessed_images = shrink(img_batch_3)  # rejects unexpected types or shapes
except ValueError as ve:
  print(ve)

Python inputs incompatible with input_signature:
  inputs: (
    tf.Tensor(
[[[0.7413678  0.62854624]
  [0.01738465 0.3431449 ]]

 [[0.51063764 0.3777541 ]
  [0.07321596 0.02137029]]], shape=(2, 2, 2), dtype=float32))
  input_signature: (
    TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name=None)).


##AutoGraph Control Flow

If your function contains a simple for loop, what do you expect will happen?

In [33]:
@tf.function
def add_10(x):
  for i in range(10):
    x += 1
  return x

It works fine, but when we look at its graph, we find that it does not contain a loop: it just contains 10 addition operations!

In [34]:
add_10(tf.constant(0))

<tf.Tensor: shape=(), dtype=int32, numpy=10>

In [35]:
add_10.get_concrete_function(tf.constant(0)).graph.get_operations()

[<tf.Operation 'x' type=Placeholder>,
 <tf.Operation 'add/y' type=Const>,
 <tf.Operation 'add' type=AddV2>,
 <tf.Operation 'add_1/y' type=Const>,
 <tf.Operation 'add_1' type=AddV2>,
 <tf.Operation 'add_2/y' type=Const>,
 <tf.Operation 'add_2' type=AddV2>,
 <tf.Operation 'add_3/y' type=Const>,
 <tf.Operation 'add_3' type=AddV2>,
 <tf.Operation 'add_4/y' type=Const>,
 <tf.Operation 'add_4' type=AddV2>,
 <tf.Operation 'add_5/y' type=Const>,
 <tf.Operation 'add_5' type=AddV2>,
 <tf.Operation 'add_6/y' type=Const>,
 <tf.Operation 'add_6' type=AddV2>,
 <tf.Operation 'add_7/y' type=Const>,
 <tf.Operation 'add_7' type=AddV2>,
 <tf.Operation 'add_8/y' type=Const>,
 <tf.Operation 'add_8' type=AddV2>,
 <tf.Operation 'add_9/y' type=Const>,
 <tf.Operation 'add_9' type=AddV2>,
 <tf.Operation 'Identity' type=Identity>]

If you want the graph to contain a “dynamic” loop instead (i.e., one that runs when the graph is executed), you can create one manually using the `tf.while_loop()` operation, but it is not very intuitive.

In [36]:
@tf.function
def add_10(x):
  condition = lambda i, x: tf.less(i, 10)
  body = lambda i, x: (tf.add(i, 1), tf.add(x, 1))
  final_i, final_x = tf.while_loop(condition, body, [tf.constant(0), x])
  return final_x

In [37]:
add_10(tf.constant(5))

<tf.Tensor: shape=(), dtype=int32, numpy=15>

In [38]:
add_10.get_concrete_function(tf.constant(5)).graph.get_operations()

[<tf.Operation 'x' type=Placeholder>,
 <tf.Operation 'Const' type=Const>,
 <tf.Operation 'while/maximum_iterations' type=Const>,
 <tf.Operation 'while/loop_counter' type=Const>,
 <tf.Operation 'while' type=StatelessWhile>,
 <tf.Operation 'Identity' type=Identity>]

Instead, it is much simpler to use TensorFlow’s `AutoGraph` feature.AutoGraph is actually activated by default (if you ever need to turn it off, you can pass `autograph=False` to `tf.function()`).

Let’s look at the graph that gets generated if you just replace `range()` with `tf.range()`.

In [39]:
@tf.function
def add_10(x):
  for i in tf.range(10):
    x = x + 1
  return x

In [40]:
add_10.get_concrete_function(tf.constant(0)).graph.get_operations()

[<tf.Operation 'x' type=Placeholder>,
 <tf.Operation 'range/start' type=Const>,
 <tf.Operation 'range/limit' type=Const>,
 <tf.Operation 'range/delta' type=Const>,
 <tf.Operation 'range' type=Range>,
 <tf.Operation 'sub' type=Sub>,
 <tf.Operation 'floordiv' type=FloorDiv>,
 <tf.Operation 'mod' type=FloorMod>,
 <tf.Operation 'zeros_like' type=Const>,
 <tf.Operation 'NotEqual' type=NotEqual>,
 <tf.Operation 'Cast' type=Cast>,
 <tf.Operation 'add' type=AddV2>,
 <tf.Operation 'zeros_like_1' type=Const>,
 <tf.Operation 'Maximum' type=Maximum>,
 <tf.Operation 'while/maximum_iterations' type=Const>,
 <tf.Operation 'while/loop_counter' type=Const>,
 <tf.Operation 'while' type=StatelessWhile>,
 <tf.Operation 'Identity' type=Identity>]

As you can see, the graph now contains a While loop operation, as if you had called
the `tf.while_loop()` function.

##Variables and Resources in TF Functions

In TensorFlow, variables and other stateful objects, such as queues or datasets, are
called resources. TF Functions treat them with special care: any operation that reads
or updates a resource is considered stateful, and TF Functions ensure that stateful
operations are executed in the order they appear (as opposed to stateless operations,
which may be run in parallel, so their order of execution is not guaranteed).

Moreover, when you pass a resource as an argument to a TF Function, it gets passed by reference, so the function may modify it.

In [41]:
counter = tf.Variable(0)

@tf.function
def increment(counter, c=1):
  return counter.assign_add(c)

In [42]:
increment(counter)  # counter is now equal to 1

<tf.Tensor: shape=(), dtype=int32, numpy=1>

In [43]:
increment(counter)  # counter is now equal to 2

<tf.Tensor: shape=(), dtype=int32, numpy=2>

In [44]:
print(counter)

<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=2>


If you peek at the function definition, the first argument is marked as a resource:

In [45]:
function_def = increment.get_concrete_function(counter).function_def
function_def.signature.input_arg[0]

name: "counter"
type: DT_RESOURCE

It is also possible to use a `tf.Variable` defined outside of the function, without
explicitly passing it as an argument:

In [46]:
counter = tf.Variable(0)

@tf.function
def increment(c=1):
  return counter.assign_add(c)

In [47]:
increment()

<tf.Tensor: shape=(), dtype=int32, numpy=1>

In [48]:
increment()

<tf.Tensor: shape=(), dtype=int32, numpy=2>

In [49]:
function_def = increment.get_concrete_function(counter).function_def
function_def.signature.input_arg[0]

name: "c"
type: DT_RESOURCE

The TF Function will treat this as an implicit first argument, so it will actually end up
with the same signature (except for the name of the argument).

However, using global
variables can quickly become messy, so you should generally wrap variables (and
other resources) inside classes.

In [50]:
class Counter:
  def __init__(self) -> None:
    self.counter = tf.Variable(0)
  
  @tf.function
  def increment(self, c=1):
    return self.counter.assign_add(c)

In [51]:
c = Counter()
c.increment()

<tf.Tensor: shape=(), dtype=int32, numpy=1>

In [52]:
c.increment()

<tf.Tensor: shape=(), dtype=int32, numpy=2>

In [53]:
c.increment()

<tf.Tensor: shape=(), dtype=int32, numpy=3>

Now, let's check generate code.

In [54]:
@tf.function
def add_10(x):
  for i in tf.range(10):
    x += 1
  return x

In [55]:
print(tf.autograph.to_code(add_10.python_function))

def tf__add(x):
    with ag__.FunctionScope('add_10', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()

        def get_state():
            return (x,)

        def set_state(vars_):
            nonlocal x
            (x,) = vars_

        def loop_body(itr):
            nonlocal x
            i = itr
            x = ag__.ld(x)
            x += 1
        i = ag__.Undefined('i')
        ag__.for_stmt(ag__.converted_call(ag__.ld(tf).range, (10,), None, fscope), None, loop_body, get_state, set_state, ('x',), {'iterate_names': 'i'})
        try:
            do_return = True
            retval_ = ag__.ld(x)
        except:
            do_return = False
            raise
        return fscope.ret(retval_, do_return)



In [56]:
def display_tf_code(func):
  from IPython.display import display, Markdown
  if hasattr(func, "python_function"):
    func = func.python_function
  code = tf.autograph.to_code(func)
  display(Markdown('```python\n{}\n```'.format(code)))

In [57]:
display_tf_code(add_10)

```python
def tf__add(x):
    with ag__.FunctionScope('add_10', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()

        def get_state():
            return (x,)

        def set_state(vars_):
            nonlocal x
            (x,) = vars_

        def loop_body(itr):
            nonlocal x
            i = itr
            x = ag__.ld(x)
            x += 1
        i = ag__.Undefined('i')
        ag__.for_stmt(ag__.converted_call(ag__.ld(tf).range, (10,), None, fscope), None, loop_body, get_state, set_state, ('x',), {'iterate_names': 'i'})
        try:
            do_return = True
            retval_ = ag__.ld(x)
        except:
            do_return = False
            raise
        return fscope.ret(retval_, do_return)

```

##TF Functions with `tf.keras`